import & setup cơ bản

In [14]:
import pandas as pd
pd.set_option('display.max_columns',500,'float_format', '{:,.2f}'.format) #hien het cac dong, format dau phay, 2 so 0
pd.set_option('display.max_rows', 500)

ĐỌC BÁO CÁO TỪ FILE CLEAN

In [15]:
path_bctc_clean_quarterly = '../../data/clean/vietstock_data/bctc_hsx_hnx_upcom/quarterly/'
file_BCTC = path_bctc_clean_quarterly + 'BCTC_quarterly.xlsx'
file_BCTC_none_bds_airline_bank = path_bctc_clean_quarterly + "BCTC_none_bds_airline_bank_quarterly.xlsx"
BCTC = pd.read_excel(file_BCTC,sheet_name=0, header=0)
#BCTC_none_bds_airline_bank = pd.read_excel(file_BCTC_none_bds_airline_bank,sheet_name=0, header=0)

Trả lời câu hỏi diễn biến lợi nhuận của toàn thị trường

In [16]:
BCTC['EBITDA'] = BCTC['15. Profit before tax (50=30+40+41)'] + BCTC['Depreciation of fixed assets and properties investment'] -  BCTC['7. Financial expenses'] # thêm EBITDA
sector_exclude = ['Real Estate Development','Scheduled Air Transportation','Depository Credit Intermediation']

BCTC_none_bds_airline_bank = BCTC[~BCTC['Sector (NAICS) Level 3'].isin(sector_exclude)] 

In [17]:
BCTC.groupby(['Year_Q'])[["TOTAL ASSETS (280=100+200)","B. OWNER'S EQUITY (400=410+430)","1. Owner's capital ",'15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)','EBITDA']].sum()


,TOTAL ASSETS (280=100+200),B. OWNER'S EQUITY (400=410+430),1. Owner's capital,15. Profit before tax (50=30+40+41),18. Net profit after tax (60 = 50 - 51 - 52),EBITDA
Year_Q,,,,,,
2022Q1,"17,061,754,859,618,572.00","3,303,448,020,754,485.00","1,832,332,588,658,724.00","167,554,637,205,274.00","137,496,180,122,131.00","99,562,898,968,871.00"
2022Q2,"17,763,170,158,378,828.00","3,395,409,171,499,892.00","1,885,428,899,664,621.00","161,219,577,858,968.00","131,189,404,956,425.00","97,154,242,257,189.00"
2022Q3,"18,146,061,038,797,520.00","3,491,522,744,356,578.00","1,932,695,202,106,154.00","147,288,945,251,359.00","113,315,898,421,246.00","83,294,521,866,385.00"
2022Q4,"18,771,099,926,967,740.00","3,548,301,992,739,428.00","1,994,482,818,845,189.00","111,745,689,218,404.00","82,519,690,137,457.00","58,907,941,291,338.00"
2023Q1,"19,086,837,142,340,072.00","3,634,866,865,056,832.00","1,992,933,910,497,387.00","140,518,688,265,171.00","109,156,282,100,540.00","73,538,955,299,799.00"
2023Q2,"19,421,493,763,761,860.00","3,689,912,667,407,475.00","2,017,442,668,263,154.00","144,705,974,360,624.00","113,091,264,591,339.00","81,385,357,886,433.00"
2023Q3,"19,828,429,226,969,488.00","3,785,187,860,388,553.00","2,078,870,391,853,954.00","138,835,350,341,863.00","107,264,128,586,273.00","65,776,532,816,106.00"
2023Q4,"21,030,537,527,000,124.00","3,880,740,434,052,606.00","2,129,959,464,807,399.00","141,766,043,990,823.00","112,748,459,352,408.00","75,290,296,602,996.00"
2024Q1,"21,174,986,011,275,804.00","3,994,688,084,942,986.00","2,129,061,292,313,091.00","145,179,102,049,220.00","116,078,602,526,532.00","67,896,584,234,435.00"


In [18]:
BCTC['Market_cap'] = BCTC["1. Owner's capital "]*BCTC['Close Price End of period']/10000
Exchange_BCTC = BCTC.groupby(['Exchange','Year_Q'])[["TOTAL ASSETS (280=100+200)","B. OWNER'S EQUITY (400=410+430)","1. Owner's capital ",'15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)','Market_cap']].sum()
Exchange_BCTC['EPS'] = Exchange_BCTC['18. Net profit after tax  (60 = 50 - 51 - 52)']/(Exchange_BCTC["1. Owner's capital "]/10000)
Exchange_BCTC['PE_quarter'] = Exchange_BCTC['Market_cap']/(Exchange_BCTC['18. Net profit after tax  (60 = 50 - 51 - 52)'])
Exchange_BCTC['Rolling_Profit_4Q'] = Exchange_BCTC.groupby('Exchange')['18. Net profit after tax  (60 = 50 - 51 - 52)'].rolling(4).sum().reset_index(level=0, drop=True)
Exchange_BCTC['Last_Year_Rolling_Profit_4Q'] = Exchange_BCTC.groupby(['Exchange'])['Rolling_Profit_4Q'].shift(4)  #lay loi nhuan rolling  cung ky nam truoc
Exchange_BCTC['YoY_Growth_Rolling_Profit_4Q'] = (Exchange_BCTC['Rolling_Profit_4Q']/Exchange_BCTC['Last_Year_Rolling_Profit_4Q']-1)*100
Exchange_BCTC['ROE(%)'] = Exchange_BCTC['Rolling_Profit_4Q']*100/Exchange_BCTC["1. Owner's capital "]
Exchange_BCTC['ROA(%)'] = Exchange_BCTC['Rolling_Profit_4Q']*100/Exchange_BCTC["TOTAL ASSETS (280=100+200)"]
Exchange_BCTC["Leverage"] = Exchange_BCTC["TOTAL ASSETS (280=100+200)"]/Exchange_BCTC["B. OWNER'S EQUITY (400=410+430)"]
Exchange_BCTC['Rolling_EPS_4Q'] = Exchange_BCTC.groupby('Exchange')['EPS'].rolling(4).sum().reset_index(level=0, drop=True)
Exchange_BCTC['Rolling_PE_4Q'] = Exchange_BCTC['Market_cap']/Exchange_BCTC['Rolling_Profit_4Q']
Exchange_BCTC

TOTAL ASSETS (280=100+200)  B. OWNER'S EQUITY (400=410+430)  \
Exchange Year_Q                                                                
HNX      2022Q1      663,873,396,686,762.00           213,560,249,460,540.00   
         2022Q2      673,732,816,880,137.00           215,997,026,588,739.00   
         2022Q3      679,192,290,945,222.00           220,427,913,266,414.00   
         2022Q4      701,310,886,809,281.00           225,971,574,483,550.00   
         2023Q1      701,313,648,175,627.00           228,300,014,208,221.00   
         2023Q2      704,917,168,995,262.00           227,907,497,989,355.00   
         2023Q3      745,819,821,049,330.00           242,078,494,289,033.00   
         2023Q4      768,885,982,070,875.00           246,399,072,376,082.00   
         2024Q1      770,586,723,025,712.00           251,386,959,411,251.00   
HOSE     2022Q1   14,682,103,454,681,530.00         2,520,039,370,150,320.00   
         2022Q2   15,333,101,229,013,080.00         2,595,293,448,670,623.00   
         2022Q3   15,718,301,245,877,618.00         2,679,842,350,339,762.00   
         2022Q4   16,340,138,326,399,836.00         2,730,865,385,944,652.00   
         2023Q1   16,696,579,281,913,784.00         2,809,948,174,133,605.00   
         2023Q2   16,965,506,591,017,116.00         2,866,643,275,443,103.00   
         2023Q3   17,291,244,633,025,956.00         2,929,050,704,694,491.00   
         2023Q4   18,392,993,003,093,656.00         3,011,169,390,540,611.00   
         2024Q1   18,561,584,916,060,008.00         3,111,192,062,182,978.00   
UPCOM    2022Q1    1,715,778,008,250,279.00           569,848,401,143,625.00   
         2022Q2    1,756,336,112,485,612.00           584,118,696,240,530.00   
         2022Q3    1,748,567,501,974,682.00           591,252,480,750,402.00   
         2022Q4    1,729,650,713,758,623.00           591,465,032,311,226.00   
         2023Q1    1,688,944,212,250,660.00           596,618,676,715,006.00   
         2023Q2    1,751,070,003,749,481.00           595,361,893,975,017.00   
         2023Q3    1,791,364,772,894,199.00           614,058,661,405,029.00   
         2023Q4    1,868,658,541,835,595.00           623,171,971,135,913.00   
         2024Q1    1,842,814,372,190,083.00           632,109,063,348,757.00   

                     1. Owner's capital   15. Profit before tax (50=30+40+41)  \
Exchange Year_Q                                                                 
HNX      2022Q1   128,555,038,621,729.00                 7,216,462,299,893.00   
         2022Q2   133,421,873,420,000.00                 6,455,456,774,879.00   
         2022Q3   140,022,487,073,000.00                 4,613,336,530,290.00   
         2022Q4   143,267,213,799,157.00                 5,828,427,622,700.00   
         2023Q1   143,217,787,910,000.00                 4,874,058,245,684.00   
         2023Q2   143,820,079,380,000.00                 6,218,255,708,156.00   
         2023Q3   156,048,368,050,800.00                 4,626,003,397,309.00   
         2023Q4   157,762,416,520,800.00                 6,641,833,958,970.00   
         2024Q1   158,308,247,459,502.00                 7,008,886,959,760.00   
HOSE     2022Q1 1,275,106,234,077,831.00               138,806,776,715,937.00   
         2022Q2 1,323,859,767,657,831.00               121,096,670,885,669.00   
         2022Q3 1,362,737,539,977,831.00               125,399,107,675,511.00   
         2022Q4 1,423,485,424,214,242.00                93,315,852,638,188.00   
         2023Q1 1,428,146,090,727,832.00               118,376,423,694,284.00   
         2023Q2 1,447,694,286,177,831.00               121,411,077,438,875.00   
         2023Q3 1,488,801,243,767,831.00               112,523,965,224,554.00   
         2023Q4 1,530,523,486,917,810.00               113,895,408,730,510.00   
         2024Q1 1,529,944,975,677,831.00               119,212,152,881,205.00   
UPCOM    2022Q1   428,671,315,959,164.00                21,531,398,189,444.0

In [19]:
BCTC_none_bds_airline_bank.groupby(['Year_Q'])[['3. Net revenue  (10 = 01 - 03)','7. Financial expenses','15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)']].sum()

,3. Net revenue (10 = 01 - 03),7. Financial expenses,15. Profit before tax (50=30+40+41),18. Net profit after tax (60 = 50 - 51 - 52)
Year_Q,,,,
2022Q1,"852,904,639,212,014.00","19,770,493,300,958.00","89,469,156,129,903.00","76,398,454,392,451.00"
2022Q2,"947,039,852,382,620.00","29,444,725,155,152.00","91,412,098,605,911.00","77,819,459,768,871.00"
2022Q3,"875,381,037,987,572.00","26,226,935,799,992.00","60,391,103,291,241.00","49,321,555,396,875.00"
2022Q4,"908,067,066,148,106.00","30,792,772,655,340.00","43,101,057,700,596.00","33,335,329,826,388.00"
2023Q1,"767,733,406,833,067.00","26,810,761,290,119.00","52,196,667,638,358.00","42,227,607,457,710.00"
2023Q2,"831,819,815,873,068.00","28,051,785,681,326.00","63,737,803,317,899.00","51,633,983,884,826.00"
2023Q3,"831,827,545,118,846.00","29,066,065,336,967.00","58,984,884,603,369.00","48,022,218,539,538.00"
2023Q4,"904,422,583,020,785.00","24,831,147,795,037.00","65,684,572,896,851.00","53,702,294,071,763.00"
2024Q1,"811,814,775,333,555.00","22,643,077,353,688.00","61,699,831,805,790.00","50,179,206,347,350.00"


TRẢ LỜI CHO CÂU HỎI THỊ TRƯỜNG NONE BANK, REAL ESTATE, AIRLINE

In [20]:
BCTC_none_bds_airline_bank['Market_cap'] = BCTC_none_bds_airline_bank["1. Owner's capital "]*BCTC_none_bds_airline_bank['Close Price End of period']/10000
Exchange_BCTC_None = BCTC_none_bds_airline_bank.groupby(['Exchange','Year_Q'])[["TOTAL ASSETS (280=100+200)","B. OWNER'S EQUITY (400=410+430)","1. Owner's capital ","III. Short-term receivables","IV. Inventories",'15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)','EBITDA','Market_cap',"Net cash flows from operating activities","Net cash flows from investing activities","Net cash flows from financing activities"]].sum()
Exchange_BCTC_None['EPS'] = Exchange_BCTC_None['18. Net profit after tax  (60 = 50 - 51 - 52)']/(Exchange_BCTC_None["1. Owner's capital "]/10000)
Exchange_BCTC_None['PE_quarter'] = Exchange_BCTC_None['Market_cap']/(Exchange_BCTC_None['18. Net profit after tax  (60 = 50 - 51 - 52)'])
Exchange_BCTC_None['Rolling_Profit_4Q'] = Exchange_BCTC_None.groupby('Exchange')['18. Net profit after tax  (60 = 50 - 51 - 52)'].rolling(4).sum().reset_index(level=0, drop=True)
Exchange_BCTC_None['Rolling_PE_4Q'] = Exchange_BCTC_None['Market_cap']/Exchange_BCTC_None['Rolling_Profit_4Q']
Exchange_BCTC_None['Last_Year_Rolling_Profit_4Q'] = Exchange_BCTC_None.groupby(['Exchange'])['Rolling_Profit_4Q'].shift(4)  #lay loi nhuan rolling  cung ky nam truoc
Exchange_BCTC_None['YoY_Growth_Rolling_Profit_4Q'] = (Exchange_BCTC_None['Rolling_Profit_4Q']/Exchange_BCTC_None['Last_Year_Rolling_Profit_4Q']-1)*100
Exchange_BCTC_None['ROE(%)'] = Exchange_BCTC_None['Rolling_Profit_4Q']*100/Exchange_BCTC_None["1. Owner's capital "]
Exchange_BCTC_None['ROA(%)'] = Exchange_BCTC_None['Rolling_Profit_4Q']*100/Exchange_BCTC_None["TOTAL ASSETS (280=100+200)"]
Exchange_BCTC_None["Leverage"] = Exchange_BCTC_None["TOTAL ASSETS (280=100+200)"]/Exchange_BCTC_None["B. OWNER'S EQUITY (400=410+430)"]

Exchange_BCTC_None

C:\Users\khoi.ln\AppData\Local\Temp\ipykernel_26672\3043153505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BCTC_none_bds_airline_bank['Market_cap'] = BCTC_none_bds_airline_bank["1. Owner's capital "]*BCTC_none_bds_airline_bank['Close Price End of period']/10000


TOTAL ASSETS (280=100+200)  B. OWNER'S EQUITY (400=410+430)  \
Exchange Year_Q                                                                
HNX      2022Q1      433,165,036,763,814.00           180,858,622,663,242.00   
         2022Q2      432,589,695,933,622.00           183,248,985,408,500.00   
         2022Q3      435,392,197,213,879.00           185,791,367,385,272.00   
         2022Q4      441,261,330,172,538.00           190,081,767,756,358.00   
         2023Q1      443,770,827,612,879.00           191,647,177,700,040.00   
         2023Q2      441,254,970,318,703.00           190,628,689,631,697.00   
         2023Q3      461,364,582,917,943.00           202,251,215,600,838.00   
         2023Q4      470,943,463,956,578.00           205,767,713,588,154.00   
         2024Q1      476,029,324,663,180.00           210,022,069,131,522.00   
HOSE     2022Q1    2,712,133,969,396,321.00         1,179,732,782,818,930.00   
         2022Q2    2,775,210,762,166,458.00         1,217,823,543,156,432.00   
         2022Q3    2,758,478,397,661,709.00         1,236,457,605,034,579.00   
         2022Q4    2,748,942,932,774,403.00         1,246,107,066,766,933.00   
         2023Q1    2,766,925,722,095,427.00         1,266,993,737,468,085.00   
         2023Q2    2,825,769,372,959,979.00         1,280,860,494,377,406.00   
         2023Q3    2,819,420,414,502,405.00         1,275,920,261,663,673.00   
         2023Q4    2,836,133,811,235,337.00         1,266,360,869,156,994.00   
         2024Q1    2,902,761,918,879,155.00         1,303,408,734,609,078.00   
UPCOM    2022Q1    1,080,850,277,515,437.00           498,456,268,379,800.00   
         2022Q2    1,102,251,326,905,101.00           511,836,706,902,646.00   
         2022Q3    1,085,531,319,355,943.00           518,089,077,134,210.00   
         2022Q4    1,076,965,331,623,282.00           524,858,650,186,213.00   
         2023Q1    1,058,825,377,583,138.00           529,125,927,476,567.00   
         2023Q2    1,073,191,383,691,283.00           525,955,165,298,981.00   
         2023Q3    1,101,173,043,977,792.00           538,490,935,848,324.00   
         2023Q4    1,112,220,495,643,916.00           542,124,484,475,372.00   
         2024Q1    1,109,568,387,552,730.00           549,447,194,263,781.00   

                   1. Owner's capital   III. Short-term receivables  \
Exchange Year_Q                                                       
HNX      2022Q1 105,335,176,751,729.00       105,319,413,352,992.00   
         2022Q2 110,189,413,710,000.00       109,986,946,886,715.00   
         2022Q3 113,873,299,683,000.00       111,293,639,124,411.00   
         2022Q4 116,667,176,369,157.00       113,439,971,020,960.00   
         2023Q1 116,612,815,480,000.00       113,039,095,731,353.00   
         2023Q2 116,958,494,160,000.00       114,627,773,083,224.00   
         2023Q3 126,498,148,980,800.00       118,333,741,120,124.00   
         2023Q4 127,667,388,400,800.00       119,669,999,335,543.00   
         2024Q1 127,588,149,339,502.00       118,486,290,100,927.00   
HOSE     2022Q1 612,202,906,707,831.00       410,754,991,187,276.00   
         2022Q2 649,254,347,767,831.00       432,293,070,266,327.00   
         2022Q3 676,216,932,897,831.00       428,026,090,849,404.00   
         2022Q4 687,567,784,534,242.00       422,858,665,464,516.00   
         2023Q1 689,279,879,127,831.00       452,138,314,704,908.00   
         2023Q2 692,912,361,307,831.00       459,548,379,662,200.00   
         2023Q3 699,212,845,467,831.00       440,781,967,238,360.00   
         2023Q4 696,751,293,417,810.00       398,728,659,886,614.00   
         2024Q1 700,121,501,957,831.00       431,339,418,384,740.00   
UPCOM    2022Q1 374,785,650,749,164.00       217,769,796,249,359.00   
         2022Q2 375,161,593,376,790.00       240,359,393,292,944.00   
         2022Q3 375,967,609,925,323.00       229,366,155,963,337.00   
         2022Q4 380,849,093,771,790.00       221,289,889,038,39

ĐÁNH GIÁ PE THEO NGÀNH

In [39]:
Sector_BCTC = BCTC.groupby(['Sector - ICB Level 4','Year_Q'])[['Stock code',"TOTAL ASSETS (280=100+200)","B. OWNER'S EQUITY (400=410+430)","1. Owner's capital ","III. Short-term receivables","IV. Inventories","3. Net revenue  (10 = 01 - 03)","5. Gross profit (20 = 10 - 11)",'EBITDA',"7. Financial expenses","9. Selling expenses","10. General and administrative expenses",'11. Operating profit {30 = 20 + (21 - 22) - (24 + 25)}','18. Net profit after tax  (60 = 50 - 51 - 52)','Market_cap',"Net cash flows from operating activities","Net cash flows from investing activities","Net cash flows from financing activities"]].sum().reset_index()
Sector_BCTC['Oprating_profit_margin'] = Sector_BCTC['11. Operating profit {30 = 20 + (21 - 22) - (24 + 25)}']*100/(Sector_BCTC["3. Net revenue  (10 = 01 - 03)"])
Sector_BCTC['EPS_quarter'] = Sector_BCTC['18. Net profit after tax  (60 = 50 - 51 - 52)']/(Sector_BCTC["1. Owner's capital "]/10000)
Sector_BCTC['Rolling_EPS_4Q'] = Sector_BCTC.groupby('Sector - ICB Level 4')['EPS_quarter'].rolling(4).sum().reset_index(level=0, drop=True)
Sector_BCTC['PE_quarter'] = Sector_BCTC['Market_cap']/(Sector_BCTC['18. Net profit after tax  (60 = 50 - 51 - 52)'])
Sector_BCTC['Rolling_Profit_4Q'] = Sector_BCTC.groupby('Sector - ICB Level 4')['18. Net profit after tax  (60 = 50 - 51 - 52)'].rolling(4).sum().reset_index(level=0, drop=True)
Sector_BCTC['Last_Year_Rolling_Profit_4Q'] = Sector_BCTC.groupby(['Sector - ICB Level 4'])['Rolling_Profit_4Q'].shift(4)  #lay loi nhuan rolling  cung ky nam truoc
Sector_BCTC['YoY_Growth_Rolling_Profit_4Q'] = (Sector_BCTC['Rolling_Profit_4Q']/Sector_BCTC['Last_Year_Rolling_Profit_4Q']-1)*100
Sector_BCTC['ROE(%)'] = Sector_BCTC['Rolling_Profit_4Q']*100/Sector_BCTC["1. Owner's capital "]
Sector_BCTC['ROA(%)'] = Sector_BCTC['Rolling_Profit_4Q']*100/Sector_BCTC["TOTAL ASSETS (280=100+200)"]
Sector_BCTC['Rolling_PE_4Q'] = Sector_BCTC['Market_cap']/Sector_BCTC['Rolling_Profit_4Q']
path_bctc_clean_quarterly = '../../data/clean/vietstock_data/bctc_hsx_hnx_upcom/quarterly/'
Sector_BCTC.to_excel(path_bctc_clean_quarterly+"Sector_BCTC.xlsx", merge_cells=True) # merge_cells=True giúp xuất các cột bị Merge như Sector - ICB Level 4, và Year_Q
Sector_BCTC
# Nhóm dữ liệu theo 'Sector - ICB Level 4' và 'Year_Q'


,Sector - ICB Level 4,Year_Q,Stock code,TOTAL ASSETS (280=100+200),B. OWNER'S EQUITY (400=410+430),1. Owner's capital,III. Short-term receivables,IV. Inventories,3. Net revenue (10 = 01 - 03),5. Gross profit (20 = 10 - 11),EBITDA,7. Financial expenses,9. Selling expenses,10. General and administrative expenses,11. Operating profit {30 = 20 + (21 - 22) - (24 + 25)},18. Net profit after tax (60 = 50 - 51 - 52),Market_cap,Net cash flows from operating activities,Net cash flows from investing activities,Net cash flows from financing activities,Oprating_profit_margin,EPS_quarter,Rolling_EPS_4Q,PE_quarter,Rolling_Profit_4Q,Last_Year_Rolling_Profit_4Q,YoY_Growth_Rolling_Profit_4Q,ROE(%),ROA(%),Rolling_PE_4Q
0,Airlines,2022Q1,HVNSCSVJC,"126,780,632,294,048.00","16,387,347,677,999.00","28,139,412,080,000.00","34,078,139,050,334.00","3,579,673,119,388.00","16,388,283,187,248.00","-1,646,393,327,834.00","-2,491,815,627,857.00","931,289,992,775.00","493,621,094,898.00","524,259,236,438.00","-2,291,098,867,133.00","-2,253,854,094,649.00","140,597,216,479,600.00","-691,409,198,855.00","-735,040,009,560.00","3,274,941,386,161.00",-13.98,-800.96,NaN,-62.38,NaN,NaN,NaN,NaN,NaN,NaN
1,Airlines,2022Q2,HVNSCSVJC,"132,704,705,312,336.00","13,954,616,013,492.00","28,139,412,080,000.00","33,432,193,212,240.00","5,359,660,601,958.00","30,122,435,461,308.00","917,003,450,179.00","-3,056,470,576,834.00","1,924,431,751,571.00","894,017,436,947.00","577,236,419,806.00","-2,038,244,960,943.00","-2,233,983,551,242.00","113,926,978,707,000.00","2,012,756,385,853.00","1,521,360,574,197.00","-2,628,234,212,850.00",-6.77,-793.90,NaN,-51.00,NaN,NaN,NaN,NaN,NaN,NaN
2,Airlines,2022Q3,HVNSCSVJC,"131,739,722,784,580.00","11,550,452,271,775.00","28,570,746,900,000.00","41,498,189,052,632.00","4,463,599,185,123.00","32,957,541,850,230.00","690,269,523,635.00","-3,192,852,363,656.00","1,698,697,645,776.00","993,429,240,055.00","656,902,362,328.00","-2,219,275,218,979.00","-2,343,117,177,512.00","97,287,903,159,100.00","-630,159,692,004.00","938,228,060,421.00","-1,521,167,772,438.00",-6.73,-820.11,NaN,-41.52,NaN,NaN,NaN,NaN,NaN,NaN
3,Airlines,2022Q4,HVNSCSVJC,"129,280,961,496,158.00","5,475,832,975,520.00","28,570,746,900,000.00","31,902,269,050,062.00","3,857,164,325,918.00","31,473,952,193,550.00","-4,505,932,433,395.00","-5,878,345,097,030.00","2,376,219,400,188.00","1,487,547,796,258.00","713,560,787,899.00","-6,122,090,192,649.00","-4,787,130,571,834.00","97,666,708,741,600.00","4,397,645,698,879.00","-1,542,849,045,902.00","-3,688,478,466,290.00",-19.45,"-1,675.54","-4,090.50",-20.40,"-11,618,085,395,237.00",NaN,NaN,-40.66,-8.99,-8.41
4,Airlines,2023Q1,HVNSCSVJC,"130,535,851,324,312.00","6,470,006,948,051.00","28,575,746,900,000.00","34,853,090,309,534.00","4,390,940,618,769.00","36,554,172,108,254.00","3,144,080,625,046.00","1,047,853,107,556.00","1,252,743,685,315.00","1,467,968,429,622.00","650,078,533,542.00","430,769,816,181.00","248,587,643,568.00","92,173,294,615,100.00","1,617,502,553,503.00","-103,813,506,783.00","-1,054,855,418,085.00",1.18,86.99,"-3,202.55",370.79,"-9,115,643,657,020.00",NaN,NaN,-31.90,-6.98,-10.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,Water,2023Q1,BDWBGWBNWBTWBWEBWSCLWCMWCTWDNADNNDNWDWCDWSGDWG...,"49,125,156,222,140.00","27,682,048,380,846.00","18,085,954,507,403.00","3,274,089,500,579.00","1,917,820,334,727.00","5,500,809,696,139.00","1,864,283,691,368.00","1,072,025,108,636.00","209,951,568,440.00","534,562,532,271.00","401,269,514,858.00","894,978,371,444.00","796,633,638,173.00","37,162,913,856,941.36","776,839,561,340.00","-660,103,174,613.00","-7,882,650,147.00",16.27,440.47,"1,828.71",46.65,"3,310,900,521,887.00",NaN,NaN,18.31,6.74,11.22
715,Water,2023Q2,BDWBGWBNWBTWBWEBWSCLWCMWCTWDNADNNDNWDWCDWSGDWG...,"50,702,423,107,306.00","27,884,493,615,432.00","18,115,608,507,403.00","3,315,456,933,055.00","2,003,716,550,416.00","6,299,743,231,698.0

In [41]:
# Sắp xếp DataFrame theo 'Sector - ICB Level 4' và 'Year_Q'
BCTC_sorted = BCTC.sort_values(['Sector - ICB Level 4', 'Year_Q'])

# Nhóm dữ liệu theo 'Sector - ICB Level 4' và 'Year_Q'
grouped = BCTC_sorted.groupby(['Sector - ICB Level 4', 'Year_Q'])['Stock code'].apply(list).reset_index()


# Tạo một hàm để tìm các 'stock code' bị thiếu so với quý trước
def find_missing_codes(group):
    prev_stock_codes = None
    missing_codes_list = []
    for index, row in group.iterrows():
        current_stock_codes = set(row['Stock code'])
        if prev_stock_codes is None:
            missing_codes_list.append('')
        else:
            missing_codes = prev_stock_codes - current_stock_codes
            missing_codes_list.append(','.join(missing_codes))
        prev_stock_codes = current_stock_codes
    group['Missing Stock Codes'] = missing_codes_list
    return group

# Áp dụng hàm cho mỗi nhóm 'Sector - ICB Level 4'
missing_codes = grouped.groupby('Sector - ICB Level 4').apply(find_missing_codes).reset_index(drop=True)
missing_codes
Sector_BCTC = Sector_BCTC.merge(
    missing_codes[['Sector - ICB Level 4', 'Year_Q', 'Missing Stock Codes']],
    on=['Sector - ICB Level 4', 'Year_Q'],
    how='left'
)
Sector_BCTC

,Sector - ICB Level 4,Year_Q,Stock code,TOTAL ASSETS (280=100+200),B. OWNER'S EQUITY (400=410+430),1. Owner's capital,III. Short-term receivables,IV. Inventories,3. Net revenue (10 = 01 - 03),5. Gross profit (20 = 10 - 11),EBITDA,7. Financial expenses,9. Selling expenses,10. General and administrative expenses,11. Operating profit {30 = 20 + (21 - 22) - (24 + 25)},18. Net profit after tax (60 = 50 - 51 - 52),Market_cap,Net cash flows from operating activities,Net cash flows from investing activities,Net cash flows from financing activities,Oprating_profit_margin,EPS_quarter,Rolling_EPS_4Q,PE_quarter,Rolling_Profit_4Q,Last_Year_Rolling_Profit_4Q,YoY_Growth_Rolling_Profit_4Q,ROE(%),ROA(%),Rolling_PE_4Q,Missing Stock Codes
0,Airlines,2022Q1,HVNSCSVJC,"126,780,632,294,048.00","16,387,347,677,999.00","28,139,412,080,000.00","34,078,139,050,334.00","3,579,673,119,388.00","16,388,283,187,248.00","-1,646,393,327,834.00","-2,491,815,627,857.00","931,289,992,775.00","493,621,094,898.00","524,259,236,438.00","-2,291,098,867,133.00","-2,253,854,094,649.00","140,597,216,479,600.00","-691,409,198,855.00","-735,040,009,560.00","3,274,941,386,161.00",-13.98,-800.96,NaN,-62.38,NaN,NaN,NaN,NaN,NaN,NaN,
1,Airlines,2022Q2,HVNSCSVJC,"132,704,705,312,336.00","13,954,616,013,492.00","28,139,412,080,000.00","33,432,193,212,240.00","5,359,660,601,958.00","30,122,435,461,308.00","917,003,450,179.00","-3,056,470,576,834.00","1,924,431,751,571.00","894,017,436,947.00","577,236,419,806.00","-2,038,244,960,943.00","-2,233,983,551,242.00","113,926,978,707,000.00","2,012,756,385,853.00","1,521,360,574,197.00","-2,628,234,212,850.00",-6.77,-793.90,NaN,-51.00,NaN,NaN,NaN,NaN,NaN,NaN,
2,Airlines,2022Q3,HVNSCSVJC,"131,739,722,784,580.00","11,550,452,271,775.00","28,570,746,900,000.00","41,498,189,052,632.00","4,463,599,185,123.00","32,957,541,850,230.00","690,269,523,635.00","-3,192,852,363,656.00","1,698,697,645,776.00","993,429,240,055.00","656,902,362,328.00","-2,219,275,218,979.00","-2,343,117,177,512.00","97,287,903,159,100.00","-630,159,692,004.00","938,228,060,421.00","-1,521,167,772,438.00",-6.73,-820.11,NaN,-41.52,NaN,NaN,NaN,NaN,NaN,NaN,
3,Airlines,2022Q4,HVNSCSVJC,"129,280,961,496,158.00","5,475,832,975,520.00","28,570,746,900,000.00","31,902,269,050,062.00","3,857,164,325,918.00","31,473,952,193,550.00","-4,505,932,433,395.00","-5,878,345,097,030.00","2,376,219,400,188.00","1,487,547,796,258.00","713,560,787,899.00","-6,122,090,192,649.00","-4,787,130,571,834.00","97,666,708,741,600.00","4,397,645,698,879.00","-1,542,849,045,902.00","-3,688,478,466,290.00",-19.45,"-1,675.54","-4,090.50",-20.40,"-11,618,085,395,237.00",NaN,NaN,-40.66,-8.99,-8.41,
4,Airlines,2023Q1,HVNSCSVJC,"130,535,851,324,312.00","6,470,006,948,051.00","28,575,746,900,000.00","34,853,090,309,534.00","4,390,940,618,769.00","36,554,172,108,254.00","3,144,080,625,046.00","1,047,853,107,556.00","1,252,743,685,315.00","1,467,968,429,622.00","650,078,533,542.00","430,769,816,181.00","248,587,643,568.00","92,173,294,615,100.00","1,617,502,553,503.00","-103,813,506,783.00","-1,054,855,418,085.00",1.18,86.99,"-3,202.55",370.79,"-9,115,643,657,020.00",NaN,NaN,-31.90,-6.98,-10.11,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,Water,2023Q1,BDWBGWBNWBTWBWEBWSCLWCMWCTWDNADNNDNWDWCDWSGDWG...,"49,125,156,222,140.00","27,682,048,380,846.00","18,085,954,507,403.00","3,274,089,500,579.00","1,917,820,334,727.00","5,500,809,696,139.00","1,864,283,691,368.00","1,072,025,108,636.00","209,951,568,440.00","534,562,532,271.00","401,269,514,858.00","894,978,371,444.00","796,633,638,173.00","37,162,913,856,941.36","776,839,561,340.00","-660,103,174,613.00","-7,882,650,147.00",16.27,440.47,"1,828.71",46.65,"3,310,900,521,887.00",NaN,NaN,18.31,6.74,11.22,
715,Water,2023Q2,BDWBGWBNWBTWBWEBWSCLWCMWCTWDNADNNDNWDWCDWSGDWG...,"50,702,423,107,306.00","27,884,493,615,432.00","18,115,608,507,403.00","3,315,456,933,055.00","2,003,716,55

Chọn Sector

In [22]:
Choose_sector = "Clothing & Accessories"
Choose_sector_overview = Sector_BCTC.loc[[Choose_sector]]
Choose_sector_overview

TOTAL ASSETS (280=100+200)  \
Sector - ICB Level 4   Year_Q                               
Clothing & Accessories 2022Q1       75,685,858,749,901.00   
                       2022Q2       80,002,836,637,146.00   
                       2022Q3       80,209,002,501,526.00   
                       2022Q4       77,728,984,392,937.00   
                       2023Q1       75,047,760,948,634.00   
                       2023Q2       75,473,397,267,785.00   
                       2023Q3       75,642,673,640,662.00   
                       2023Q4       76,037,558,714,302.00   
                       2024Q1       75,580,358,521,983.00   

                               B. OWNER'S EQUITY (400=410+430)  \
Sector - ICB Level 4   Year_Q                                    
Clothing & Accessories 2022Q1            32,343,015,663,637.00   
                       2022Q2            33,787,760,647,246.00   
                       2022Q3            35,316,267,844,162.00   
                       2022Q4            35,299,196,791,593.00   
                       2023Q1            35,337,769,244,625.00   
                       2023Q2            34,341,724,120,014.00   
                       2023Q3            34,793,795,440,111.00   
                       2023Q4            35,218,804,895,080.00   
                       2024Q1            34,483,037,082,039.00   

                                1. Owner's capital   \
Sector - ICB Level 4   Year_Q                         
Clothing & Accessories 2022Q1 15,745,965,970,000.00   
                       2022Q2 16,537,790,630,000.00   
                       2022Q3 17,227,568,130,000.00   
                       2022Q4 17,505,243,050,001.00   
                       2023Q1 17,575,020,060,000.00   
                       2023Q2 17,683,731,970,000.00   
                       2023Q3 18,079,698,470,000.00   
                       2023Q4 18,158,464,340,000.00   
                       2024Q1 18,308,131,859,502.00   

                               III. Short-term receivables  \
Sector - ICB Level 4   Year_Q                                
Clothing & Accessories 2022Q1        16,048,213,797,060.00   
                       2022Q2        16,656,070,841,599.00   
                       2022Q3        15,331,034,753,886.00   
                       2022Q4        13,666,818,011,325.00   
                       2023Q1        13,680,861,911,717.00   
                       2023Q2        14,365,342,094,557.00   
                       2023Q3        15,235,437,121,523.00   
                       2023Q4        15,543,281,189,359.00   
                       2024Q1        14,444,115,446,510.00   

                                    IV. Inventories  \
Sector - ICB Level 4   Year_Q                         
Clothing & Accessories 2022Q1 15,957,167,332,337.00   
                       2022Q2 17,832,293,076,875.00   
                       2022Q3 17,470,260,328,348.00   
                       2022Q4 17,928,070,567,894.00   
                       2023Q1 17,442,419,945,848.00   
                       2023Q2 17,594,052,724,396.00   
                       2023Q3 15,461,732,512,959.00   
                       2023Q4 15,689,593,732,027.00   
                       2024Q1 16,198,702,945,657.00   

                               3. Net revenue  (10 = 01 - 03)  \
Sector - ICB Level 4   Year_Q                                   
Clothing & Accessories 2022Q1           20,049,665,609,580.00   
                       2022Q2           22,106,443,805,421.00   
                       2022Q3           21,458,698,347,081.00   
                       2022Q4           19,546,593,085,391.00   
                       2023Q1           16,950,880,689,763.00   
                       2023Q2           19,158,086,078,592.00   
                       2023Q3           19,446,985,483,700.00   
                       2023Q4           19,192,521,957,087.00   
                       2024Q1           17,286,784,792,260.00   

                     

In [23]:
Choose_sector_detailview = BCTC[BCTC['Sector - ICB Level 4']==Choose_sector][["Stock code","Year_Q","TOTAL ASSETS (280=100+200)","1. Owner's capital ",'18. Net profit after tax  (60 = 50 - 51 - 52)','Market_cap']]
Choose_sector_detailview['Last_Year_Quarter_Profit'] = Choose_sector_detailview.groupby(["Stock code"])['18. Net profit after tax  (60 = 50 - 51 - 52)'].shift(4)  #lay loi nhuan quy cung ky nam truoc
Choose_sector_detailview['YoY_Growth_Quarter'] = (Choose_sector_detailview['18. Net profit after tax  (60 = 50 - 51 - 52)']/Choose_sector_detailview['Last_Year_Quarter_Profit']-1)*100
Choose_sector_detailview['EPS_quarter'] = Choose_sector_detailview['18. Net profit after tax  (60 = 50 - 51 - 52)']/(Choose_sector_detailview["1. Owner's capital "]/10000)
Choose_sector_detailview['Rolling_EPS_4Q'] = Choose_sector_detailview.groupby('Stock code')['EPS_quarter'].rolling(4).sum().reset_index(level=0, drop=True)
Choose_sector_detailview['Rolling_Profit_4Q'] = Choose_sector_detailview.groupby('Stock code')['18. Net profit after tax  (60 = 50 - 51 - 52)'].rolling(4).sum().reset_index(level=0, drop=True)
Choose_sector_detailview['Last_Year_Rolling_Profit_4Q'] = Choose_sector_detailview.groupby(["Stock code"])['Rolling_Profit_4Q'].shift(4)  #lay loi nhuan rolling  cung ky nam truoc
Choose_sector_detailview['YoY_Growth_Rolling_Profit_4Q'] = (Choose_sector_detailview['Rolling_Profit_4Q']/Choose_sector_detailview['Last_Year_Rolling_Profit_4Q']-1)*100
Choose_sector_detailview['Rolling_PE_4Q'] = Choose_sector_detailview['Market_cap']/Choose_sector_detailview['Rolling_Profit_4Q']
Choose_sector_detailview['ROE(%)'] = Choose_sector_detailview['Rolling_Profit_4Q']*100/Choose_sector_detailview["1. Owner's capital "]
Choose_sector_detailview['ROA(%)'] = Choose_sector_detailview['Rolling_Profit_4Q']*100/Choose_sector_detailview["TOTAL ASSETS (280=100+200)"]
Choose_sector_detailview

,Stock code,Year_Q,TOTAL ASSETS (280=100+200),1. Owner's capital,18. Net profit after tax (60 = 50 - 51 - 52),Market_cap,Last_Year_Quarter_Profit,YoY_Growth_Quarter,EPS_quarter,Rolling_EPS_4Q,Rolling_Profit_4Q,Last_Year_Rolling_Profit_4Q,YoY_Growth_Rolling_Profit_4Q,Rolling_PE_4Q,ROE(%),ROA(%)
27,AAT,2022Q1,"1,237,793,386,803.00","638,014,890,000.00","21,407,935.00","874,080,399,300.00",NaN,NaN,0.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,AAT,2022Q2,"1,099,972,930,564.00","638,014,890,000.00","88,324,143,919.00","593,353,847,700.00",NaN,NaN,"1,384.36",NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,AAT,2022Q3,"1,059,538,048,227.00","638,014,890,000.00","610,784,187.00","606,114,145,500.00",NaN,NaN,9.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,AAT,2022Q4,"1,056,917,551,882.00","638,014,890,000.00","1,369,341,229.00","316,455,385,440.00",NaN,NaN,21.46,"1,415.73","90,325,677,270.00",NaN,NaN,3.50,14.16,8.55
31,AAT,2023Q1,"1,045,033,628,753.00","638,014,890,000.00","1,465,256,966.00","317,731,415,220.00","21,407,935.00","6,744.46",22.97,"1,438.36","91,769,526,301.00",NaN,NaN,3.46,14.38,8.78
32,AAT,2023Q2,"1,025,553,158,528.00","638,014,890,000.00","1,672,764,138.00","403,863,425,370.00","88,324,143,919.00",-98.11,26.22,80.22,"5,118,146,520.00",NaN,NaN,78.91,0.80,0.50
33,AAT,2023Q3,"1,011,228,368,206.00","708,191,030,000.00","654,592,502.00","368,259,335,600.00","610,784,187.00",7.17,9.24,79.89,"5,161,954,835.00",NaN,NaN,71.34,0.73,0.51
34,AAT,2023Q4,"1,053,155,592,171.00","708,191,030,000.00","9,745,595,398.00","347,013,604,700.00","1,369,341,229.00",611.70,137.61,196.04,"13,538,209,004.00","90,325,677,270.00",-85.01,25.63,1.91,1.29
35,AAT,2024Q1,"1,047,016,432,570.00","708,191,030,000.00","5,108,234,011.00","314,436,817,320.00","1,465,256,966.00",248.62,72.13,245.20,"17,181,186,049.00","91,769,526,301.00",-81.28,18.30,2.43,1.64
189,ADS,2022Q1,"2,289,053,692,567.00","380,694,500,000.00","28,528,753,613.00","1,406,666,177,500.00",NaN,NaN,749.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN
